In [9]:
import nest_asyncio
from dotenv import load_dotenv
from IPython.display import Markdown, display
import re
from nltk.corpus import stopwords
import nltk
from nltk.translate.meteor_score import meteor_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import PromptTemplate
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, ServiceContext, SimpleDirectoryReader

In [10]:
questions =[
    {
        "question":"What is the definition of an accident according to the contract?",
        "ground_truth": "Sudden, unforeseen, and violent event caused by an external factor that results in injuries requiring medical attention to the insured within 90 days subsequent to the event."
    },
    {
        "question": "What is covered during a stay at the hospital?",
        "ground_truth": "So, according to this information, the following are covered during a stay at the hospital: Private standard room with bathroom, Food provided, Laboratory tests, cabinet, image, and diagnostic agents related to the diagnosis of the covered accident or disease Operating room, healing, recovery, emergency, intensive care, intermediate, or coronary care unit, Blood transfusions and blood compatibility tests, Medications administered in the hospital that comply with the definition in these general conditions, are indicated and justified by the attending physician, and are related to the covered accident or disease, Expenses for an additional bed for the policyholder's companion affected and General nursing care"
    },
    {
        "question": "What rehabilitation therapy is covered by the insurance?",
        "ground_truth": "Rehabilitation musculoskeletal (30 sessions), Hydrotherapy (30 sessions), Neurorehabilitation (180 sessions), Therapy for motor disorders (90 sessions), exclusively for newborns with AXA benefit, Speech therapy (30 sessions), Inhalation therapy or pulmonary physiotherapy, as indicated by the attending physician, Cardiac rehabilitation (40 sessions)"
    },
    {
        "question": "What is the coverage for accidents that need dental work??",
        "ground_truth": "Consultation of emergency dental care at normal hours once a year without cost, Diagnosis, treatment plan, and prevention, 2 cleanings per year without cost (for adults and children), Radiographs (1 radiograph per year) with 30% co-pay, Dental operation (amalgams and resins) with 30% co-pay, Endodontics (root canal treatments, pulpectomies, and pulpotomias) with 30% co-pay and a 10% discount for other treatments, Minor surgery (minor surgical procedures) with 30% co-pay, Major surgery (major surgical procedures) with a 15% discount, Third molars (wisdom teeth) with a 20% discount, Prosthetics (crowns, onlays, posts) with a 10% discount, Periodontology (1 periodontal evaluation per year without cost) with 30% co-pay and a 10% discount for other treatments, Orthodontics (brackets) with a 10-20% discount for bracket removal, Whitening (Zoom and at-home) with a 20% discount"
    },
    {
        "question": "What is the policy regarding surgery by robot?",
        "ground_truth":"The policy covers surgery by robot for specific procedures, including prostatectomy, hysterectomy due to endometrial cancer, ovarian cancer, cervical-uterine cancer, renal cancer, and others. However, only surgeons with expertise in minimally invasive surgery and recertification in the relevant specialty will be covered, provided they have performed at least 20 robotic surgeries for that specific procedure."

    },
    {
        "question": "What is the coverage for COVID-19?",
        "ground_truth":"For the insured with a policy effective date after February 9, 2022, COVID-19 medical expenses are fully covered if they have completed the complete vaccination schedule as per the National Vaccination Plan. For those without the complete vaccination schedule, COVID-19 medical expenses are partially covered up to $300,000."
    },
    {
        "question": "What is the coverage for medication outside the hospital?",
        "ground_truth":"Medicamentos Fuera del Hospital (MFH): Medications purchased outside the hospital are covered, even if other additional coverages such as Emergencia Médica en el Extranjero (EMER), Enfermedades Cubiertas en el Extranjero (ECE), or Atención en el Extranjero (ATEX) are contracted."
    },
    {
        "question":"What is your deductible for an accident?",
        "ground_truth":"$0 (zero pesos), as per the policy conditions, after the accidental coverage has been contracted. The deductible will only be applicable if the total cost of the accident is less than $2,000."

    },
    {
        "question":"What is the coverage if you are out of country?",
        "ground_truth":"Enfermedades Cubiertas en el Extranjero (ECE), if you are out of country, this coverage provides medical expenses for certain diseases such as cancer and cerebrovascular/cerebral diseases, but only if the diagnosis and initial costs occurred within Mexico."
    },
    {
        "question":"What is Planmed?",
        "ground_truth":"Planmed is a medical coverage offered by AXA Flex Plus 2024, providing benefits such as Check Up, Consultas de Atención Primaria (Primary Care), Atención no programada (Non-programmed Attention), and Consultas de Especialidad (Specialty Consultations)."
    },
    {
        "question":"What are the rules if I get injured in a racecar?",
        "ground_truth": "According to section 35 of the policy, injuries sustained by the insured while traveling as an occupant (driver or passenger) of a vehicle, including a racecar, during racing, endurance, or speed tests, are not covered."
    },
    {
        "question":"What is the coverage for a heart attack?",
        "ground_truth":"Coverage includes surgical intervention for coronary heart disease, provided the need for such interventions is demonstrated through coronary angiography."
    },
    {
        "question":"What is the coverage for a heart transplant?",
        "ground_truth":"The coverage for a heart transplant includes: The surgical procedure, Hospitalization expenses (as per the contracted table), Donor-related expenses, including those incurred by the donor during the donation process, Expenses related to procurement and transportation are not covered."
    },
    {
        "question":"What is the coverage for a kidney transplant?",
        "ground_truth":"This policy covers the procedure for kidney transplantation, but excludes post-surgical costs. Additionally, it also covers the donor's medical expenses if they are alive during the donation process, and only the recipient's expenses if the donor is deceased."
    },
    {
        "question":"What is the policy on hyperbaric medicine?",
        "ground_truth":"According to the conditions, any costs related to Hyperbaric Treatments are not covered, unless specified under coverage 16."
    }
    
]

In [11]:
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))

def tokenize(text):
    tokens = re.findall(r'\b\w+\b', text.lower())
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return filtered_tokens

def calculate_f1(predicted_answer, ground_truth_answer):
    pred_tokens = set(tokenize(predicted_answer))
    gt_tokens = set(tokenize(ground_truth_answer))

    common_tokens = pred_tokens.intersection(gt_tokens)
    if len(common_tokens) == 0:
        return 0.0

    precision = len(common_tokens) / len(pred_tokens)
    recall = len(common_tokens) / len(gt_tokens)
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

def rouge_1(predicted_answer, ground_truth_answer):
    pred_tokens = tokenize(predicted_answer)
    gt_tokens = tokenize(ground_truth_answer)
    overlapping_tokens = set(pred_tokens).intersection(gt_tokens)
    return len(overlapping_tokens) / len(gt_tokens) if gt_tokens else 0

def cosine_sim(predicted_answer, ground_truth_answer):
    vectorizer = CountVectorizer().fit([predicted_answer, ground_truth_answer])
    vectors = vectorizer.transform([predicted_answer, ground_truth_answer])
    return cosine_similarity(vectors)[0, 1]

def jaccard_similarity(predicted_answer, ground_truth_answer):
    pred_tokens = set(tokenize(predicted_answer))
    gt_tokens = set(tokenize(ground_truth_answer))
    intersection = len(pred_tokens.intersection(gt_tokens))
    union = len(pred_tokens.union(gt_tokens))
    return intersection / union if union != 0 else 0

def calculate_meteor(predicted_answer, ground_truth_answer):
    gt_tokens = tokenize(ground_truth_answer)
    pred_tokens = tokenize(predicted_answer)
    return meteor_score([gt_tokens], pred_tokens)


[nltk_data] Downloading package stopwords to C:\Users\SPARTAN
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\SPARTAN
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
nest_asyncio.apply()

In [ ]:
input_dir_path = './Docs/'

In [14]:
llm=Ollama(model="llama3.1:8b", request_timeout=120.0)
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2", trust_remote_code=True)

In [15]:
# load data
loader = SimpleDirectoryReader(
            input_dir = input_dir_path,
            recursive=True
        )
docs = loader.load_data()
Settings.embed_model = embed_model
index = VectorStoreIndex.from_documents(docs, show_progress=True)

Settings.llm = llm
query_engine = index.as_query_engine()

Generating embeddings: 100%|██████████| 137/137 [00:02<00:00, 55.75it/s]


In [16]:
qa_prompt_tmpl_str = (
"Context information is below.\n"
"---------------------\n"
"{context_str}\n"
"---------------------\n"
"Given the context information above I want you to give clear and concise answers. Translate EVERY word from Spanish to English, including procedures, and do not write the Spanish words.\n"
"Query: {query_str}\n"
"Answer: "
)
qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

f1_scores = []
rouge_scores = []
cosine_scores = []
jaccard_scores = []
meteor_scores = []
average_f1 = []
average_rouge = []
average_cosine = []
average_jaccard = []
average_meteor = []
for i in range(50):
    for qa_pair in questions:
        question = qa_pair["question"]
        ground_truth = qa_pair["ground_truth"]
        
        # Get the AI's answer for the current question
        predicted_answer = query_engine.query(question,)
        
        # Calculate the F1 score for this pair
        f1 = calculate_f1(str(predicted_answer), ground_truth)
        rogue = rouge_1(str(predicted_answer), ground_truth)
        cosine = cosine_sim(str(predicted_answer), ground_truth)
        jaccard = jaccard_similarity(str(predicted_answer), ground_truth)
        meteor = calculate_meteor(str(predicted_answer), ground_truth)
        f1_scores.append(f1)
        rouge_scores.append(rogue)
        cosine_scores.append(cosine)
        jaccard_scores.append(jaccard)
        meteor_scores.append(meteor)

        # Print the F1 score for this question-answer pair
        print(f"Question: {question}")
        print(f"Ground Truth: {ground_truth}")
        print(f"Predicted Answer: {predicted_answer}")
        print(f"F1 Score: {f1}\n")
        print(f"Rouge Score: {rogue}\n")
        print(f"Cosine Similarity: {cosine}\n")
        print(f"Jaccard Similarity: {jaccard}\n")   
        print(f"Meteor Score: {meteor}\n")
    

    average_f1.append(sum(f1_scores) / len(f1_scores))
    average_rouge.append(sum(rouge_scores) / len(rouge_scores))
    average_cosine.append(sum(cosine_scores) / len(cosine_scores))
    average_jaccard.append(sum(jaccard_scores) / len(jaccard_scores))
    average_meteor.append(sum(meteor_scores) / len(meteor_scores))


print("Average F1 Score over 50 runs:", sum(average_f1)/ len(average_f1))
print("Average Rouge Score over 50 runs:", sum(average_rouge)/ len(average_rouge))
print("Average Cosine Similarity over 50 runs:", sum(average_cosine)/ len(average_cosine))
print("Average Jaccard Similarity over 50 runs:", sum(average_jaccard)/ len(average_jaccard))
print("Average Meteor Score over 50 runs:", sum(average_meteor)/ len(average_meteor))




Question: What is the definition of an accident according to the contract?
Ground Truth: Sudden, unforeseen, and violent event caused by an external factor that results in injuries requiring medical attention to the insured within 90 days subsequent to the event.
Predicted Answer: According to the contract, an Accident is defined as a sudden, unexpected, and violent occurrence caused by an external factor that results in injuries requiring medical attention to the Insured within 90 days following the event.
F1 Score: 0.7692307692307693

Rouge Score: 0.8333333333333334

Cosine Similarity: 0.8303988737722913

Jaccard Similarity: 0.625

Meteor Score: 0.8074879227053138

Question: What is covered during a stay at the hospital?
Ground Truth: So, according to this information, the following are covered during a stay at the hospital: Private standard room with bathroom, Food provided, Laboratory tests, cabinet, image, and diagnostic agents related to the diagnosis of the covered accident or d